# 這堂課的任務
  * Step 0: 引入會用到的套件
  * Step 1: 下載語料檔 (語料已經過斷詞前處理)
  * Step 2: 將語料讀進來
  * Step 3: 觀察語料
  * Step 4: 對語料進行 Indexing
  * Step 5: 實作 Concordance
  * Step 6: 實作 Collocation

> 迷路時的提醒：請記得真實世界中的例子，你現在手上有一本小說 (Step 0 - 3)，小說的最後還沒有任何Index，所以我們首先要進行 Indexing (Step 4)，產生出 Index 後 ，我們要利用這個 Index 做查詢 (Step 5, 6)。

# Step 0: 引入會用到的套件

In [1]:
# 用來讀取以json格式儲存的語料檔 [Step 2]
import json

# defaultdict 用來方便進行 indexing
# Counter 用來方便統計各版文章數
# [Step 3, 4]
from collections import defaultdict, Counter 

# 用來計算程式執行時間 [Step 5]
import time

# 用來對 token 做 regular expression 查詢 [Step 5]
import re

# 用來顯示美美的查詢結果 [Step 5]
from tabulate import tabulate
import pandas as pd
from IPython.display import HTML, display
# %load_ext google.colab.data_table [Step 5 用來顯示互動式dataframe]

# Step 1: 下載語料檔

In [2]:
# 用來下載 Google Drive 文件的 Python 套件
# Colab 已內建此指令
!pip install gdown

In [3]:
!gdown --id "1mmAdISiVBw_acDRiyoAFFOIJoyDQ6dPa" -O "dcard2.jsonl"

Downloading...
From: https://drive.google.com/uc?id=1mmAdISiVBw_acDRiyoAFFOIJoyDQ6dPa
To: /content/dcard2.jsonl
178MB [00:00, 183MB/s]


In [4]:
# TODO: 看 dcard2.jsonl 的前五行
!head -5 dcard2.jsonl

{"title": "#問 Dr. Martens 1460 鞋墊", "commentCount": 14, "likeCount": 3, "forumName": "穿搭", "gender": 0, "text": [[{"word": "雖然", "pos": "Cbb"}, {"word": "知道", "pos": "VK"}, {"word": "可能", "pos": "D"}, {"word": "會", "pos": "D"}, {"word": "被", "pos": "P"}, {"word": "噴", "pos": "VC"}, {"word": "，", "pos": "COMMACATEGORY"}, {"word": "可是", "pos": "Cbb"}, {"word": "最近", "pos": "Nd"}, {"word": "想", "pos": "VE"}, {"word": "直接", "pos": "VH"}, {"word": "從", "pos": "P"}, {"word": "國外", "pos": "Nc"}, {"word": "網站", "pos": "Nc"}, {"word": "買", "pos": "VC"}, {"word": "都", "pos": "D"}, {"word": "沒有", "pos": "VJ"}, {"word": "UK4", "pos": "Nb"}, {"word": "的", "pos": "DE"}, {"word": "貨", "pos": "Na"}, {"word": "缺貨", "pos": "VH"}, {"word": "了", "pos": "Di"}, {"word": "好", "pos": "VH"}, {"word": "一陣子", "pos": "Nd"}, {"word": "甚至", "pos": "D"}, {"word": "連", "pos": "VJ"}, {"word": "選項", "pos": "Na"}, {"word": "都", "pos": "D"}, {"word": "消失", "pos": "VA"}, {"word": "，", "pos": "COMMACATEGORY"}, {"word": "專櫃

<a href="https://drive.google.com/file/d/1Yp4501Db7Dyj_gJg0f107FnTDg2ohhhw/view?usp=sharing" _target="blank"><img src="https://drive.google.com/uc?id=1Yp4501Db7Dyj_gJg0f107FnTDg2ohhhw" width="400"></a>

觀察：我們的語料中，有什麼 metadata 和 data?

# Step 2: 將語料讀進來

In [5]:
RAW_DATA_PATH = "dcard2.jsonl"

# 用一個空的 list 準備裝進語料
corpus = []

# TODO: 將每行 json 一一讀進 corpus:
with open(RAW_DATA_PATH, "r", encoding="utf-8") as file:
    
    # 對一個 file 進行 for loop ，就是一次處理該檔案一行的內容
    # 所以變數 article 在每一次迴圈中就是語料檔中的每一行
    for article in file:
        
        # 將每一行的 json 字串讀進來
        article_json = json.loads(article)

        # 依序把解讀出來的每一行加進 corpus
        corpus.append(article_json)

<a href="https://drive.google.com/file/d/1KR9XvKZS1sgGtS_Eqo91-vAUOdDrkh5W/view?usp=sharing" _target="blank"><img src="https://drive.google.com/uc?id=1KR9XvKZS1sgGtS_Eqo91-vAUOdDrkh5W" width="900"></a>


# Step 3: 觀察語料

## 3-0: 認識語料的資料結構

In [6]:
# TODO: 查看總文章篇數
# 因為 corpus 是一個 list，我們剛剛把每篇文章一一加入這個 list
# 所以用 len() 去讀取這個 list 的長度，就是去看我們總共有幾篇文章
len(corpus)

19224

In [7]:
# TODO: 查看語料中的第零篇文章內容
corpus[0]

{'commentCount': 14,
 'date': '2020-01-13',
 'forumName': '穿搭',
 'gender': 0,
 'likeCount': 3,
 'text': [[{'pos': 'Cbb', 'word': '雖然'},
   {'pos': 'VK', 'word': '知道'},
   {'pos': 'D', 'word': '可能'},
   {'pos': 'D', 'word': '會'},
   {'pos': 'P', 'word': '被'},
   {'pos': 'VC', 'word': '噴'},
   {'pos': 'COMMACATEGORY', 'word': '，'},
   {'pos': 'Cbb', 'word': '可是'},
   {'pos': 'Nd', 'word': '最近'},
   {'pos': 'VE', 'word': '想'},
   {'pos': 'VH', 'word': '直接'},
   {'pos': 'P', 'word': '從'},
   {'pos': 'Nc', 'word': '國外'},
   {'pos': 'Nc', 'word': '網站'},
   {'pos': 'VC', 'word': '買'},
   {'pos': 'D', 'word': '都'},
   {'pos': 'VJ', 'word': '沒有'},
   {'pos': 'Nb', 'word': 'UK4'},
   {'pos': 'DE', 'word': '的'},
   {'pos': 'Na', 'word': '貨'},
   {'pos': 'VH', 'word': '缺貨'},
   {'pos': 'Di', 'word': '了'},
   {'pos': 'VH', 'word': '好'},
   {'pos': 'Nd', 'word': '一陣子'},
   {'pos': 'D', 'word': '甚至'},
   {'pos': 'VJ', 'word': '連'},
   {'pos': 'Na', 'word': '選項'},
   {'pos': 'D', 'word': '都'},
   {'po

In [8]:
# TODO: 查看第零篇文章的版名
corpus[0]['forumName']

'穿搭'

In [9]:
# TODO: 查看第零篇文章中的第零個句子
corpus[0]['text'][0]

[{'pos': 'Cbb', 'word': '雖然'},
 {'pos': 'VK', 'word': '知道'},
 {'pos': 'D', 'word': '可能'},
 {'pos': 'D', 'word': '會'},
 {'pos': 'P', 'word': '被'},
 {'pos': 'VC', 'word': '噴'},
 {'pos': 'COMMACATEGORY', 'word': '，'},
 {'pos': 'Cbb', 'word': '可是'},
 {'pos': 'Nd', 'word': '最近'},
 {'pos': 'VE', 'word': '想'},
 {'pos': 'VH', 'word': '直接'},
 {'pos': 'P', 'word': '從'},
 {'pos': 'Nc', 'word': '國外'},
 {'pos': 'Nc', 'word': '網站'},
 {'pos': 'VC', 'word': '買'},
 {'pos': 'D', 'word': '都'},
 {'pos': 'VJ', 'word': '沒有'},
 {'pos': 'Nb', 'word': 'UK4'},
 {'pos': 'DE', 'word': '的'},
 {'pos': 'Na', 'word': '貨'},
 {'pos': 'VH', 'word': '缺貨'},
 {'pos': 'Di', 'word': '了'},
 {'pos': 'VH', 'word': '好'},
 {'pos': 'Nd', 'word': '一陣子'},
 {'pos': 'D', 'word': '甚至'},
 {'pos': 'VJ', 'word': '連'},
 {'pos': 'Na', 'word': '選項'},
 {'pos': 'D', 'word': '都'},
 {'pos': 'VA', 'word': '消失'},
 {'pos': 'COMMACATEGORY', 'word': '，'},
 {'pos': 'Nc', 'word': '專櫃'},
 {'pos': 'VH', 'word': '直接'},
 {'pos': 'VC', 'word': '買'},
 {'pos'

In [10]:
# TODO: 查看第零篇文章中的第零個句子中的第零個token
corpus[0]['text'][0][0]

{'pos': 'Cbb', 'word': '雖然'}

In [11]:
# TODO: 查看第零篇文章中的第零個句子中的第零個token的詞目是什麼
corpus[0]['text'][0][0]['word']

'雖然'

<a href="https://drive.google.com/file/d/1SCPNqwhftuRLHI7jd-l7xEc620kZRjNC/view?usp=sharing" _target="blank"><img src="https://drive.google.com/uc?id=1SCPNqwhftuRLHI7jd-l7xEc620kZRjNC" width="900"></a>


`corpus` 是我們用來儲存語料的地方，由於我們可以餵位置給`corpus`，讓`corpus`告訴我們該位置的詞是什麼。在這個意義下，`corpus`其實就是一個 Foward Index (正向索引)。

## 3-1: 觀察 metadata

In [12]:
# TODO: 看我們的語料中到底有哪些版，以及每個版各有幾篇文章
#       請讓 forum 這個變數的內容最後的結果如下圖
# 我們將以 collections.Counter 來達成這個目標
# 使用前要先引入: from collections import Counter
forum = Counter()

# 請接著完成版名的統計
for article in corpus:
    forum[article['forumName']] += 1

<a href="https://drive.google.com/file/d/17WIpNn8m_7UPyjHXOSlJ331v51Jtknzm/view?usp=sharing" _target="blank"><img src="https://drive.google.com/uc?id=17WIpNn8m_7UPyjHXOSlJ331v51Jtknzm" width="400"></a>

In [13]:
# TODO: 看一下 forum 長什麼樣子
forum

Counter({'3C': 215,
         'App': 210,
         'Apple': 216,
         'BL': 209,
         'BTS': 219,
         'Les': 219,
         'Netflix': 89,
         'YouTuber': 211,
         '信用卡': 215,
         '健身': 208,
         '僑生交流': 107,
         '動漫': 223,
         '台中': 2,
         '咖啡': 181,
         '品酒': 197,
         '塔羅': 223,
         '女孩': 222,
         '實習職缺': 212,
         '寵物': 234,
         '寶可夢': 217,
         '居家生活': 193,
         '工作': 229,
         '廢文': 202,
         '建議回饋': 208,
         '彩虹': 212,
         '心情': 218,
         '感情': 216,
         '成功大學': 205,
         '戲劇綜藝': 173,
         '手作': 235,
         '手搖': 75,
         '打工職缺': 214,
         '攝影': 234,
         '文具': 195,
         '新生季': 211,
         '旅遊': 241,
         '日劇': 77,
         '日本生活': 167,
         '星座': 202,
         '時事': 210,
         '書籍': 217,
         '有趣': 219,
         '歐美影集': 156,
         '汽機車': 213,
         '淡江大學': 212,
         '減肥': 219,
         '港澳 u life': 205,
         '港澳女生': 

In [14]:
# TODO: 叫出前十多文章的版
# 請使用 Counter.most_common()
forum.most_common(10)

[('靈異', 244),
 ('美妝', 242),
 ('旅遊', 241),
 ('精品', 240),
 ('網路購物', 238),
 ('穿搭', 236),
 ('手作', 235),
 ('寵物', 234),
 ('攝影', 234),
 ('工作', 229)]

# Step 4: 對語料進行 Indexing (重頭戲🎪)

想像你是一個機器人，現在我丟給你一本斷好詞的「哈利波特：阿茲卡班的逃犯」，

請你幫我產生一個可以附在最後面的索引，好讓讀者可以去查「妙麗」出現的地方。

這時你會怎麼做？

## 4-0: 暖身操: 利用 `enumerate()` 取得list中的位置訊息

In [15]:
s = ["今天", "天氣", "晴"]

for element in s:
    print(element)

今天
天氣
晴


In [16]:
# TODO: 請使用 enumerate() 在印出 list 當中的每個元素的同時，也一併印出該元素的 index
for element_index, element in enumerate(s):
    print(f"{element_index} {element}")

0 今天
1 天氣
2 晴


In [17]:
# 假設我們今天有一個更複雜的語料結構（與我們的語料結構雷同）

fake_corpus = [
    # article 0
    [
        # setence 0
        [
            {"word": "這", "pos": "NH"},     # token 0
            {"word": "是", "pos": "SHI"},    # token 1
            {"word": "第", "pos": "N"},      # token 2
            {"word": "零", "pos": "X"},      # ...
            {"word": "句", "pos": "X"},
         
        ],
        # sentence 1
        [
            {"word": "這", "pos": "NH"},
            {"word": "是", "pos": "SHI"},
            {"word": "第", "pos": "N"},
            {"word": "一", "pos": "X"},
            {"word": "句", "pos": "X"},
        ],
    ],

    # article 1
    [
        # sentence 0
        [
            {"word": "另", "pos": "NH"},
            {"word": "一", "pos": "SHI"},
            {"word": "篇", "pos": "N"},
            {"word": "文章", "pos": "X"},
        ],
    ]
]

In [18]:
# TODO: 請用迴圈取出每一個token的詞目(word)，讓輸出的每行是一個單獨的token詞目
# 範例輸出:
# 這
# 是
# 第
# 零
# 句
# 這
# 是
# 第
# 一
# 句
# 另
# 一
# 篇
# 文章

for article in fake_corpus:
  for sentence in article:
    for token in sentence:
        print(token["word"])

這
是
第
零
句
這
是
第
一
句
另
一
篇
文章


In [19]:
# TODO: 那要如何搭配 enumerate() 才能給出每一個詞目所在的 文章id, 句子id, token id 呢？
# 範例輸出:
# 0 0 0 這
# 0 0 1 是
# 0 0 2 第
# 0 0 3 零
# 0 0 4 句
# 0 1 0 這
# 0 1 1 是
# 0 1 2 第
# 0 1 3 一
# 0 1 4 句
# 1 0 0 另
# 1 0 1 一
# 1 0 2 篇
# 1 0 3 文章

for article_idx, article in enumerate(fake_corpus):
  for sentence_idx, sentence in enumerate(article):
    for token_idx, token in enumerate(sentence):
        print(str(article_idx) + " " + str(sentence_idx) + " " + str(token_idx) + " " + token["word"] )

0 0 0 這
0 0 1 是
0 0 2 第
0 0 3 零
0 0 4 句
0 1 0 這
0 1 1 是
0 1 2 第
0 1 3 一
0 1 4 句
1 0 0 另
1 0 1 一
1 0 2 篇
1 0 3 文章


## 4-1: 對詞目進行Indexing

In [20]:
word_index = {}    # 用來儲存詞目的索引

# TODO: 請用 4-0 所提到的方法，對 corpus 做讀取，並讓 word_index 最後的結果如下圖

# 對 corpus 中的每一行進行讀取。每一行就是一篇貼文。
for article_idx, article in enumerate(corpus):
    for sentence_idx, sentence in enumerate(article['text']):
        for token_idx, token in enumerate(sentence):
            
            # 到這裡我們已經有了現在處理到第幾篇文章 (article_idx), 第幾句 (sentence_idx), 第幾個 token (token_idx) 的訊息了
            # 我們把這三個訊息放進一個 list 當中，並取為 locator 變數名稱
            locator = [article_idx, sentence_idx, token_idx]

            # 這裡我們要判斷目前讀到的詞目，是否已經有出現在 word_index 當中
            # 如果目前還沒遇過這個詞目，那們我們要為word_index新增一組 key/value
            # key 為這個詞， value 為一個list，並把剛剛記錄下來的 locator 也順便存進去
            if token['word'] not in word_index:          
                word_index[token['word']] = [locator]

            # 如果目前處理的詞目已經有記錄在 word_index 當中了，那麼只要把 locator 加進現有的 list 當中即可
            else:
                word_index[token['word']].append(locator)

<a href="https://drive.google.com/file/d/1a7bs6Pux1CINo1ONGf_C0DW6aRXSgSkO/view?usp=sharing" _target="blank"><img src="https://drive.google.com/uc?id=1a7bs6Pux1CINo1ONGf_C0DW6aRXSgSkO" width="400"></a>

## 4-2: 換個方式對詞性做Indexing

使用 Python 提供給我們的工具: `defaultdict`

要先 import 進來才能使用:
```python
from collections import defaultdict
```

In [21]:
pos_index = defaultdict(list)     # 用來儲存詞性的索引

# 對 dcard2.jsonl 中的每一行進行讀取。每一行就是一篇貼文。
for article_idx, article in enumerate(corpus):
    for sentence_idx, sentence in enumerate(article['text']):
        for token_idx, token in enumerate(sentence):
            
            locator = [article_idx, sentence_idx, token_idx]

            # 使用 defaultdict 的方便之處在於
            # 這裡我們就不用多做 4-1 中所做的判斷
            # defaultdict 會自動幫我們處理這件事
            pos_index[token['pos']].append(locator) 

## 4-3: 查看 Indexing 後的結果

In [22]:
# TODO: 叫出「虐心」這個詞在我們語料中所有出現的地方
# 並解讀出現結果的結構/意義是什麼
word_index['虐心']

[[5777, 7, 4],
 [5915, 8, 6],
 [5939, 39, 1],
 [5965, 1, 90],
 [8369, 8, 45],
 [10506, 43, 23],
 [12021, 15, 6],
 [12223, 1, 4],
 [14607, 0, 29],
 [17410, 8, 29]]

<a href="https://drive.google.com/file/d/1nZ-h8McCSXVrHKqz-hro_dZMjoMefRn5/view?usp=sharing" _target="blank"><img src="https://drive.google.com/uc?id=1nZ-h8McCSXVrHKqz-hro_dZMjoMefRn5" width="400"></a>

In [23]:
#TODO: 寫一個函數 word_freq(word)
#      輸入一個詞，回傳該詞詞頻
def word_freq(word):
    return len(word_index[word])

In [24]:
# TODO: 請找出「的」的詞頻
word_freq("的")

231504

In [25]:
#TODO: 寫一個函數 pos_freq(pos)
#      輸入一個詞性，回傳該詞性頻率
def pos_freq(pos):
    return len(pos_index[pos])

In [26]:
# TODO: 請找出我們語料中word type的數量
len(word_index.keys())

186298

In [27]:
# TODO: 請找出我們語料中pos type的數量
len(pos_index.keys())

61

In [28]:
# TODO: 請列出所有的詞性
pos_index.keys()

dict_keys(['Cbb', 'VK', 'D', 'P', 'VC', 'COMMACATEGORY', 'Nd', 'VE', 'VH', 'Nc', 'VJ', 'Nb', 'DE', 'Na', 'Di', 'VA', 'Dfa', 'Nh', 'Neqa', 'Ng', 'Neu', 'Nf', 'VB', 'SHI', 'VL', 'T', 'FW', 'VD', 'Ncd', 'Nep', 'V_2', 'I', 'QUESTIONCATEGORY', 'Nes', 'PARENTHESISCATEGORY', 'EXCLAMATIONCATEGORY', 'A', 'PAUSECATEGORY', 'VCL', 'COLONCATEGORY', 'DASHCATEGORY', 'Nv', 'VF', 'PERIODCATEGORY', 'Caa', 'VG', 'DOTCATEGORY', 'Dk', 'VHC', 'Dfb', 'WHITESPACE', 'Cab', 'VI', 'ETCCATEGORY', 'Da', 'Cba', 'Neqb', 'DM', 'SEMICOLONCATEGORY', 'VAC', 'SPCHANGECATEGORY'])

詞性意義請參考：
- [中研院平衡語料庫詞類標記集](http://ckipsvr.iis.sinica.edu.tw/papers/category_list.pdf)
- [中研院詞庫小組. (1993). 中文詞類分析(三版)](https://ckip.iis.sinica.edu.tw/data/paper/report/ckip-9305.pdf)

In [29]:
# TODO: 請找出我們語料的總token數是多少
token_sum = 0
for word, list_of_locator in word_index.items():
    token_sum += word_freq(word)
print(token_sum)

5292615


## [插曲] 比較有無 index 的時間差

In [30]:
def compare_time_with_and_without_index(word):

    print(f"查詢的詞: {word}")
    print(f"該詞於語料庫中的詞頻: {len(word_index[word])}")


    ###### 使用 index: 給時間寶貴的大忙人 ######
    t1 = time.time()
    for a_idx, s_idx, w_idx in word_index[word]:
        pass
    t2 = time.time()
    t_with_index = t2 - t1
    

    ###### 不使用 index: 給勤奮的螞蟻 ######
    t1 = time.time()
    
    # TODO: 如果要每次搜尋都要重新掃過一整個語料庫看某詞有沒有出現
    for article in corpus:
        for sentence in article['text']:
            for token in sentence:
                if token['word'] == word:
                    pass
    
    t2 = time.time()
    t_without_index = t2 - t1

    ###### 不使用index的時間 比上 使用index的時間 ######
    ratio = t_without_index / t_with_index
    
    # 印出結果
    print(f"使用index的時間: {t_with_index:.6f}")
    print(f"不使用index的時間: {t_without_index:.6f}")
    print(f"without_index / with_index = {ratio:6f}")
    print("-----------")

In [31]:
compare_time_with_and_without_index("虐心")

查詢的詞: 虐心
該詞於語料庫中的詞頻: 10
使用index的時間: 0.000009
不使用index的時間: 0.446027
without_index / with_index = 47968.564103
-----------


In [32]:
# TODO: 試看看針對不同出現頻率的詞，使用/不使用index的時間差距，並試著解讀為什麼會有這樣的差別。
# 比較: 出現 1      次的 "珍妮"
#      出現 510    次的 "造成"
#      出現 87610  次的 "是" 
#      出現 231504 次的 "的"
compare_time_with_and_without_index("珍妮")
compare_time_with_and_without_index("造成")
compare_time_with_and_without_index("是")
compare_time_with_and_without_index("的")

查詢的詞: 珍妮
該詞於語料庫中的詞頻: 1
使用index的時間: 0.000003
不使用index的時間: 0.448261
without_index / with_index = 170922.272727
-----------
查詢的詞: 造成
該詞於語料庫中的詞頻: 510
使用index的時間: 0.000261
不使用index的時間: 0.448383
without_index / with_index = 1717.492237
-----------
查詢的詞: 是
該詞於語料庫中的詞頻: 87610
使用index的時間: 0.019850
不使用index的時間: 0.433727
without_index / with_index = 21.849662
-----------
查詢的詞: 的
該詞於語料庫中的詞頻: 231504
使用index的時間: 0.036871
不使用index的時間: 0.442067
without_index / with_index = 11.989583
-----------


為什麼一詞的詞頻越大，使用 index 的效率會越來越接近不使用 index 的效率？ 感受一下，這樣的差距與日常生活中的經驗是否一致？

# Step 5: Concordance



## 5-0: 從 word_index 中搜尋單詞出現的句子

In [33]:
# TODO: 請利用 word_index ，印出所有出現「虐心」一詞的句子
#       每個詞目之間請以一個空格隔開
for a_idx, s_idx, t_idx in word_index['虐心']:
    sentence = corpus[a_idx]["text"][s_idx]
    print(" ".join(token["word"] for token in sentence))

「 太 寫實 太 虐心 了 吧 ？ 」
看到 結局 的 地方   太 虐心 太 深刻
最 虐心 的 地方 就 是 他 與 上官曦 被 綁 在 木頭 上
熬過 將近 一 年 的 「 限古令 」 之後 ， 大 神級 IP劇 的 輪番 登場 儼然 成 了 2020 開年 以來 的 最 燦爛 風光 ， 從 《 慶餘年 》 到 《 劍王朝 》 、 《 將 夜 2 》 、 《 絕代 雙驕 》 ， 莫不是 期望值 破錶 ， 製作 頂級 ， 話題 吸睛 ， 卡斯 自 帶 光環 … ， 一時之間 說得上 鑼鼓喧天 ， 來勢洶洶 。 其中 《 三生 三世 枕 上書 》 啣接 《 三 生 三 世 十里 桃花 》 當年 讓 人 如癡如醉 的 虐心 指數 ， 從 一 年 半 前 在 橫店 開拍 ， 到 殺青 到 定檔 ， 都 是 矚目 焦點 。
最後 一 集 也 像 上 一 季 一樣 ， 有 令 人 非常 震撼 的 劇情 ， 尤其 是 克雷 的 那 段 演講 ， 真的 讓 安迪 差點 噴淚 ， 如果 那 一 段 交給 韓國 來 拍 ， 絕對 會 非常 的 虐心 ， 可能 不亞於 〖 與 神 同行〗 。 「 請 幫 漢娜 做夢 ， 別 讓 他人 奪走 你 的 夢想 」 ， 這 句 話 不但 是 對著 克雷 他們 說 ， 也 是 對著 螢幕 前 的 每 個 你 妳 你 ， 或許 有時候 我們 會 對 人生 絕望 、 對 周遭 的 朋友 失望 ， 但 請 相信 總 有 一 雙 手 在 適當 的 時 機會 伸出來 讓 你 緊握 並且 將 你 扶起 ， 如果 中途 就 放棄 了 ， 那 這 雙 手 永遠 也 不會 出現 。
能 親眼 看到 她 的 回覆 ， 或許 接下來 的 三 年 ( 甚至 更 長 . . . ) 就 不用 這麼 虐心 了
超推 ！ 劇情 緊張 刺激 又 虐心 ， 而且 第二 季 的 女 主角 也 太 正 了 😍
他們 的 愛情 好 虐心  😭
從 去年 底 才 開始 看 八 大 戲劇 TV版 的 德魯納 酒店 ， 我 是 看 重播 的 ， 看到 一半 以後 感覺 每 一 集 都 在 虐心
我 想 把 跟 妳 的 點點滴滴 寫下來 ， 有 一 天 我們 磨合 想 放棄 時 ， 我們 可以 翻 一 翻 我們 當初 選擇 在一起 時 多 虐心 哭 了 多少 回 。


## 5-1: 加進 window_size 的考量

In [34]:
# 先以「虐心」第一個出現的地方為例: [5777, 7, 4]

# TODO: 宣告一個變數 sentence，並將 sentence 設為語料中的第 5777 篇文章中的第 7 句
sentence = corpus[5777]["text"][7]

# sentence 是一個 list of tokens
print(sentence)    

[{'word': '「', 'pos': 'PARENTHESISCATEGORY'}, {'word': '太', 'pos': 'Dfa'}, {'word': '寫實', 'pos': 'VH'}, {'word': '太', 'pos': 'Dfa'}, {'word': '虐心', 'pos': 'VH'}, {'word': '了', 'pos': 'T'}, {'word': '吧', 'pos': 'T'}, {'word': '？', 'pos': 'QUESTIONCATEGORY'}, {'word': '」', 'pos': 'PARENTHESISCATEGORY'}]


In [35]:
# TODO: 請列出此句中的每個詞目為何
#       每個詞目之間請以一個空格隔開
print(" ".join(token["word"] for token in sentence))

「 太 寫實 太 虐心 了 吧 ？ 」


In [36]:
# TODO: 請印出此句中「虐心」的前 3 個詞到後 3 個詞 (window_size 為 3)
window_size = 3

for i in range(4 - window_size, 4 + window_size + 1):
  print(sentence[i]["word"] + " ", end="")

太 寫實 太 虐心 了 吧 ？ 

<a href="https://drive.google.com/file/d/1RGJzG7SHH_aBZvz0FDK1KBcFwKnSRj4l/view?usp=sharing" _target="blank"><img src="https://drive.google.com/uc?id=1RGJzG7SHH_aBZvz0FDK1KBcFwKnSRj4l" width="400"></a>

In [37]:
# TODO: 將 window_size 設為 5 看看
# 會出現錯誤 IndexError: list index out of range，為什麼？
window_size = 5
for i in range(4 - window_size, 4 + window_size + 1):
  print(sentence[i]["word"] + " ", end="")

」 「 太 寫實 太 虐心 了 吧 ？ 」 

IndexError: ignored

In [38]:
# 錯誤來自於兩個地方
#    (1) (該詞出現的位置 - window_size) 可能會小於 0: 也就是會超出句子左邊的邊界
#    (2) (該詞出現的位置 + window_size) 可能會大於 句子長度: 也就是會超出句子右邊的邊界
# 所以我們必須設停損點:
#    (1) 對左邊邊界而言，如果 (該詞出現的位置 - window_size) < 0 ，那麼就停在 0 數值 => 使用 max(0, (該詞出現的位置 - window_size))
#    (2) 對右邊邊界而言，如果 (該詞出現的位置 + window_size) > 句子長度 ，那麼就停在句子長度的數值 => min(句子長度, (該詞出現的位置 + window_size))

# TODO: 讓我們來加進安全邊界的考量吧

window_size = 100

safe_left_bound = max(0, 4 - window_size)
safe_right_bound = min(len(sentence), 4 + window_size + 1)

for i in range(safe_left_bound, safe_right_bound):
  print(sentence[i]["word"] + " ", end="")

「 太 寫實 太 虐心 了 吧 ？ 」 

## 5-2 完成第一個 `query_a_word()` 功能

能查詢單個詞目，並按照需求顯示相關資訊。

執行範例：
```python
query_a_word("虐心", window_size=5, show_pos=False, metadata=["forumName", "likeCount"])
```

會回傳
```python
[ 
     {"article_id": 30, "left": "這部戲真的很", "keyword": "虐心", "right": "！", "forumName": "戲劇", "likeCount": 20 }, # 第0句
     {"article_id": 33, "left": "真的太", "keyword": "虐心", "right": "了吧。", "forumName": "文學", "likeCount": 20 }, # 第1句
     ...
 ]
```

In [39]:
# TODO: 寫一個函式 query_a_word(word, window_size, show_pos, metadata), 接收三個參數:
#     word: 要搜尋的關鍵字 <字串>
#     window_size: 指定視窗大小 <整數>
#     show_pos: 是否顯示詞性 <Bool>
#     metadata: 需要顯示哪些 metadata <字串的list>
# 此函數將回傳所有該關鍵字出現的句子，結構為 list of dicts

def query_a_word(word, window_size=5, show_pos=False, metadata=[]):

    # 最後的結果
    results = []

    # 從索引中叫出該詞出現的地方，並用迴圈一一處理
    for a_idx, s_idx, t_idx in word_index[word]:

        # 該詞出現的某個句子
        sentence = corpus[a_idx]['text'][s_idx]

        # 準備好每個要裝進 results 的元素
        each_concordance_line = {
            "article_id": a_idx,
            "left": "",
            "keyword": "",
            "right": ""
        }

        # 需要哪些metadata，就把那些metadata裝進來
        for key in metadata:
            each_concordance_line[key] = corpus[a_idx][key]

        # 判斷關鍵字是否要顯示詞性
        if show_pos:
            each_concordance_line["keyword"] = sentence[t_idx]["word"] + "/" + sentence[t_idx]["pos"]

        else:
            each_concordance_line["keyword"] = sentence[t_idx]["word"]

        # 安全邊界
        safe_left_bound = max(0, t_idx - window_size)
        safe_right_bound = min(len(sentence), t_idx + window_size + 1)


        # 先處理處理關鍵字的左邊部分
        for token in sentence[safe_left_bound: t_idx]:
            if show_pos:
                each_concordance_line["left"] += token["word"] + "/" + token["pos"]
            else:
                each_concordance_line["left"] += token["word"]
            # 上面的寫法等價於 each_concordance_line["left"] = each_concordance_line["left"] + token["word"]


        # 再處理處理關鍵字的右邊部分
        for token in sentence[t_idx + 1: safe_right_bound]:
            if show_pos:
                each_concordance_line["right"] += token["word"] + "/" + token["pos"]
            else:
                each_concordance_line["right"] += token["word"]

        # 每次處理完後記得把 each_concordance_line 加進 results 中
        results.append(each_concordance_line)

    return results

In [40]:
# TODO: 試著使用 query_a_word() 看看
result_nuexin = query_a_word("虐心", window_size=10, show_pos=True, metadata=["forumName", "likeCount"])
result_nuexin

[{'article_id': 5777,
  'forumName': '色情漫畫',
  'keyword': '虐心/VH',
  'left': '「/PARENTHESISCATEGORY太/Dfa寫實/VH太/Dfa',
  'likeCount': 93,
  'right': '了/T吧/T？/QUESTIONCATEGORY」/PARENTHESISCATEGORY'},
 {'article_id': 5915,
  'forumName': '戲劇綜藝',
  'keyword': '虐心/Na',
  'left': '看到/VE結局/Na的/DE地方/Na /WHITESPACE太/Dfa',
  'likeCount': 148,
  'right': '太/Dfa深刻/VH'},
 {'article_id': 5939,
  'forumName': '戲劇綜藝',
  'keyword': '虐心/VH',
  'left': '最/Dfa',
  'likeCount': 124,
  'right': '的/DE地方/Na就/D是/SHI他/Nh與/P上官曦/Na被/P綁/VC在/P'},
 {'article_id': 5965,
  'forumName': '戲劇綜藝',
  'keyword': '虐心/Na',
  'left': '三/Neu世/Na十里/Nd桃花/Na》/PARENTHESISCATEGORY當年/Nd讓/VL人/Na如癡如醉/VH的/DE',
  'likeCount': 39,
  'right': '指數/Na，/COMMACATEGORY從/P一/Neu年/Nf半/Neqb前/Ng在/P橫店/Nc開拍/VC'},
 {'article_id': 8369,
  'forumName': '歐美影集',
  'keyword': '虐心/VH',
  'left': '段/Nf交給/VD韓國/Nc來/D拍/VC，/COMMACATEGORY絕對/D會/D非常/Dfa的/DE',
  'likeCount': 37,
  'right': '，/COMMACATEGORY可能/D不亞於/VJ〖/VH與/Caa神/Na同行〗/Na。/PERIODCATEGORY「/PARENTHESISCATEG

In [41]:
# TODO: 把結果餵給 pd.DataFrame()
columns_order = ['article_id', 'forumName', 'likeCount', "left", "keyword", "right"]

pd.DataFrame(result_nuexin, columns=columns_order)

,article_id,forumName,likeCount,left,keyword,right
0,5777,色情漫畫,93,「/PARENTHESISCATEGORY太/Dfa寫實/VH太/Dfa,虐心/VH,了/T吧/T？/QUESTIONCATEGORY」/PARENTHESISCATEGORY
1,5915,戲劇綜藝,148,看到/VE結局/Na的/DE地方/Na /WHITESPACE太/Dfa,虐心/Na,太/Dfa深刻/VH
2,5939,戲劇綜藝,124,最/Dfa,虐心/VH,的/DE地方/Na就/D是/SHI他/Nh與/P上官曦/Na被/P綁/VC在/P
3,5965,戲劇綜藝,39,三/Neu世/Na十里/Nd桃花/Na》/PARENTHESISCATEGORY當年/Nd讓...,虐心/Na,指數/Na，/COMMACATEGORY從/P一/Neu年/Nf半/Neqb前/Ng在/P橫...
4,8369,歐美影集,37,段/Nf交給/VD韓國/Nc來/D拍/VC，/COMMACATEGORY絕對/D會/D非常/...,虐心/VH,，/COMMACATEGORY可能/D不亞於/VJ〖/VH與/Caa神/Na同行〗/Na。/...
5,10506,彩虹,56,甚至/D更/D長/VH./PERIODCATEGORY./PERIODCATEGORY./P...,虐心/VH,了/T
6,12021,電影,128,超推/VH！/EXCLAMATIONCATEGORY劇情/Na緊張/VH刺激/VC又/Caa,虐心/Na,，/COMMACATEGORY而且/Cbb第二/Neu季/Nd的/DE女/Na主角/Na也/...
7,12223,閒聊,2,他們/Nh的/DE愛情/Na好/VH,虐心/Na,😭/FW
8,14607,戲劇綜藝,16,，/COMMACATEGORY看到/VE一半/Neqa以後/Ng感覺/VK每/Nes一/Ne...,虐心/Na,
9,17410,Les,121,可以/D翻/VC一/Di翻/VC我們/Nh當初/Nd選擇/VC在一起/VH時/Ng多/VH,虐心/Na,哭/VA了/Di多少/Neqa回/Nf。/PERIODCATEGORY


## 5-3: 重新組織一下巨大的 `query_a_word()`

<a href="https://drive.google.com/file/d/1qpiJjz6oQOF1w0tMg0O4Wk5QSVvW3VWg/view?usp=sharing" _target="blank"><img src="https://drive.google.com/uc?id=1qpiJjz6oQOF1w0tMg0O4Wk5QSVvW3VWg" width="400"></a>

`query_a_word()` 裡頭做的事，其實可以分成兩大部分:

1. 篩選index: 從 index 中篩選出符合某一個 query 條件的 locator
 - 詞目為 "虐心"
 - 詞性為 "V"
 - 多詞目並列： "做" "一" "個"
 - 詞性, 詞目並列： "V" "起來"
2. 生成一個個的concordance line: 處理顯示結果
 - 需不需要顯示詞性？
 - 要給出幾個 window_size 的上下文?
 - 需要給出哪些metadata?

區分這兩大部分的好處，有助於我們之後做更複雜的查詢時，可以更專注地處理程式的不同部分。

In [42]:
def query_a_word(word, window_size=5, show_pos=False, metadata=[]):

    results = []

    # 在這裡我們用 locator 取代原本的 a_idx, s_idx, t_idx
    # 所以現在 locator 就會是一個 list: [a_idx, s_idx, t_idx]
    # 我們把開箱 locator 的任務交給 generate_concordance_line()
    for locator in filter_a_word(word):       
        result = generate_concordance_line(locator, window_size, show_pos, metadata)
        results.append(result)

    return results


def filter_a_word(word):
    return word_index[word]


def generate_concordance_line(locator, window_size, show_pos, metadata):
    a_idx = locator[0]
    s_idx = locator[1]
    t_idx = locator[2]

    sentence = corpus[a_idx]['text'][s_idx]

    # 準備好每個元素
    each_concordance_line = {
        "article_id": a_idx,
        "left": "",
        "keyword": "",
        "right": ""
    }

    # 需要哪些metadata，就把那些metadata裝進來
    for key in metadata:
        each_concordance_line[key] = corpus[a_idx][key]

    # 判斷是否要顯示詞性
    if show_pos:
        each_concordance_line["keyword"] = sentence[t_idx]["word"] + "/" + sentence[t_idx]["pos"]
    else:
        each_concordance_line["keyword"] = sentence[t_idx]["word"]

    # 安全邊界
    safe_left_bound = max(0, t_idx - window_size)
    safe_right_bound = min(len(sentence), t_idx + window_size + 1)


    # 先處理處理關鍵字的左邊部分
    for token in sentence[safe_left_bound: t_idx]:
        if show_pos:
            each_concordance_line["left"] += token["word"] + "/" + token["pos"]
        else:
            each_concordance_line["left"] += token["word"]

    # 再處理處理關鍵字的右邊部分
    for token in sentence[t_idx + 1: safe_right_bound]:
        if show_pos:
            each_concordance_line["right"] += token["word"] + "/" + token["pos"]
        else:
            each_concordance_line["right"] += token["word"]

    return each_concordance_line


In [43]:
# 把結果餵給 pd.DataFrame()
result_nuexin = query_a_word("虐心", window_size=10, show_pos=False, metadata=["forumName", "likeCount"])
pd.DataFrame(result_nuexin, columns=columns_order)

,article_id,forumName,likeCount,left,keyword,right
0,5777,色情漫畫,93,「太寫實太,虐心,了吧？」
1,5915,戲劇綜藝,148,看到結局的地方 太,虐心,太深刻
2,5939,戲劇綜藝,124,最,虐心,的地方就是他與上官曦被綁在
3,5965,戲劇綜藝,39,三世十里桃花》當年讓人如癡如醉的,虐心,指數，從一年半前在橫店開拍
4,8369,歐美影集,37,段交給韓國來拍，絕對會非常的,虐心,，可能不亞於〖與神同行〗。「請
5,10506,彩虹,56,甚至更長...)就不用這麼,虐心,了
6,12021,電影,128,超推！劇情緊張刺激又,虐心,，而且第二季的女主角也太正
7,12223,閒聊,2,他們的愛情好,虐心,😭
8,14607,戲劇綜藝,16,，看到一半以後感覺每一集都在,虐心,
9,17410,Les,121,可以翻一翻我們當初選擇在一起時多,虐心,哭了多少回。


## 5-4: 美美的資料，閱讀舒適度提升




1. 資料與資料呈現
2.  閱讀版面舒適度調整

<a href="https://drive.google.com/file/d/1sNPMBGdCMGTiqKunsYpAw0r3BdhQQalL/view" target="_blank"><img src="https://drive.google.com/u/0/uc?id=1sNPMBGdCMGTiqKunsYpAw0r3BdhQQalL&export=download" width="85%"></a>

> 關於資料呈現的想像：直接面對海量資料就好像近視沒戴眼鏡，或是去看 3D 電影不帶特製眼鏡一樣，好像有東西在那裡卻又好模糊、不確知那是什麼或是傳達了什麼。

資料視覺化的工具 ( [Charting in Colaboratory](https://colab.research.google.com/notebooks/charts.ipynb#scrollTo=QSMmdrrVLZ-N)  )

- [Matplotlib](https://matplotlib.org/) : the most common charting package. 
- [Seaborn](http://seaborn.pydata.org/) : One of several libraries layered on top of Matplotlib that is worth highlighting, and you can use in Colab. (Colaboratory charts use Seaborn's custom styling by default.)
- [Altair](https://altair-viz.github.io/) : a declarative visualization library for creating interactive visualizations in Python, and is installed and enabled in Colab by default. 


### 5-4-0: 若是「文字資料」的呈現呢？

<img src="https://drive.google.com/u/0/uc?id=1KultyJIj4cGq6E9LhF0tHLTRo_7073E3&export=download" width="50%">


> 適合一直放在心上的探問：要怎傳達(資料)會是最有效、直覺的？

### 5-4-1: [python-tabulate](https://pypi.org/project/tabulate/) 

- 輕鬆印出表格：只要呼叫一個 function 就能完成！
- 將輕量純文本轉為表格數據：具多種輸出格式供後續編輯與轉換
- 混合的文本與數據資料也能很好讀：欄位對齊、數字格式


In [44]:
# 感覺一下使用 python-tabulate 呈現資料
from tabulate import tabulate

In [45]:
table = [["Sun",696000,1989100000],["Earth",6371,5973.6],["Moon",1737,73.5],["Mars",3390,641.85]]

print(table)
print(tabulate(table, headers=["Planet","R (km)", "mass (x 10^29 kg)"]))

[['Sun', 696000, 1989100000], ['Earth', 6371, 5973.6], ['Moon', 1737, 73.5], ['Mars', 3390, 641.85]]
Planet      R (km)    mass (x 10^29 kg)
--------  --------  -------------------
Sun         696000           1.9891e+09
Earth         6371        5973.6
Moon          1737          73.5
Mars          3390         641.85


In [46]:
print(tabulate(result_nuexin, headers="keys", showindex="always", tablefmt="simple"))

      article_id  left                              keyword    right                           forumName      likeCount
--  ------------  --------------------------------  ---------  ------------------------------  -----------  -----------
 0          5777  「太寫實太                        虐心       了吧？」                        色情漫畫              93
 1          5915  看到結局的地方 太                 虐心       太深刻                          戲劇綜藝             148
 2          5939  最                                虐心       的地方就是他與上官曦被綁在      戲劇綜藝             124
 3          5965  三世十里桃花》當年讓人如癡如醉的  虐心       指數，從一年半前在橫店開拍      戲劇綜藝              39
 4          8369  段交給韓國來拍，絕對會非常的      虐心       ，可能不亞於〖與神同行〗。「請  歐美影集              37
 5         10506  甚至更長...)就不用這麼            虐心       了                              彩虹                  56
 6         12021  超推！劇情緊張刺激又              虐心       ，而且第二季的女主角也太正      電影                 128
 7         12223  他們的愛情好                      虐心       😭                           

當資料不只有 10 筆時，看起來會怎麼樣呢？

In [47]:
# 可以的話會全數列出來，當資料比較多的時候，並不適合使用。
result_yen = query_a_word("語言", 10)

print(tabulate(result_yen, headers="keys", showindex="always", tablefmt="simple"))

       article_id  left                                                    keyword    right
---  ------------  ------------------------------------------------------  ---------  ------------------------------------------------
  0           225  不過手機                                                語言       如果是中文 他會說中文的樣子
  1           225  所以我手機                                              語言       改韓文 順便練習韓文😂
  2           235  韓、英、西、日，四種                                    語言       ，
  3           376  *                                                       語言       中心不適用此優惠 需購買一般票價
  4           754  這就像是我最近在學                                      語言       一樣
  5           754  那些方法我                                              語言       一定會進步，但就是因為懶
  6           754  導致現在                                                語言       進展部份還是一塌糊塗
  7           772  會安排團體講解，可選擇中英等                            語言       導覽，我覺得是很小巧舒適的行程
  8           774  P.S. 入口處有每個點的各國                             

### 5-4-2: [pandas](https://pandas.pydata.org/docs/user_guide/index.html)

- 讀取表格數據(tabular data)資料的超實用工具
- 資料類型：Series(一維) ＆ DataFrame(二維, 很常用)
- 直接看 [Cheat Sheet](https://pandas.pydata.org/Pandas_Cheat_Sheet.pdf) 了解更多


In [48]:
# 使用 pandas 中的 pd.DataFrame() 的結果
import pandas as pd

pd.DataFrame(result_nuexin)

,article_id,left,keyword,right,forumName,likeCount
0,5777,「太寫實太,虐心,了吧？」,色情漫畫,93
1,5915,看到結局的地方 太,虐心,太深刻,戲劇綜藝,148
2,5939,最,虐心,的地方就是他與上官曦被綁在,戲劇綜藝,124
3,5965,三世十里桃花》當年讓人如癡如醉的,虐心,指數，從一年半前在橫店開拍,戲劇綜藝,39
4,8369,段交給韓國來拍，絕對會非常的,虐心,，可能不亞於〖與神同行〗。「請,歐美影集,37
5,10506,甚至更長...)就不用這麼,虐心,了,彩虹,56
6,12021,超推！劇情緊張刺激又,虐心,，而且第二季的女主角也太正,電影,128
7,12223,他們的愛情好,虐心,😭,閒聊,2
8,14607,，看到一半以後感覺每一集都在,虐心,,戲劇綜藝,16
9,17410,可以翻一翻我們當初選擇在一起時多,虐心,哭了多少回。,Les,121


In [49]:
#TODO: 請大家使用 query_a_word() 查詢「語言」，並把結果餵給 pd.DataFrame()

result_yen = query_a_word("語言", window_size=10, show_pos=False, metadata=["forumName", "likeCount"])
pd.DataFrame(result_yen, columns=columns_order)


,article_id,forumName,likeCount,left,keyword,right
0,225,韓星,49,不過手機,語言,如果是中文 他會說中文的樣子
1,225,韓星,49,所以我手機,語言,改韓文 順便練習韓文😂
2,235,韓星,71,韓、英、西、日，四種,語言,，
3,376,省錢,403,*,語言,中心不適用此優惠 需購買一般票價
4,754,護理,197,這就像是我最近在學,語言,一樣
...,...,...,...,...,...,...
435,18820,心情,10,,語言,組織有點亂，不多說了，不
436,18950,電影,6,都處理得相當完美，藉由他的鏡頭,語言,下，觀眾經思考後，能輕鬆看
437,18959,語言,51,她小時候被診斷有,語言,表達障礙
438,18960,語言,0,我不知道這篇應該放在,語言,還是健身版


In [50]:
pd.DataFrame(query_a_word("是", window_size=10, show_pos=False, metadata=["forumName", "likeCount"]), columns=columns_order)

,article_id,forumName,likeCount,left,keyword,right
0,0,穿搭,3,大部分都符合正貨的標準，可是鞋墊居然,是,可以拔的讓我很擔心會是假
1,0,穿搭,3,居然是可以拔的讓我很擔心會,是,假的，無意間找到一篇來源不明在
2,0,穿搭,3,剛剛打電話到Dr. Martens客服時他說鞋墊,是,可以拔的，鞋底因為沒一梯不
3,0,穿搭,3,雙的照片希望有人幫我解答到底,是,不是正貨🙇
4,0,穿搭,3,照片希望有人幫我解答到底是不,是,正貨🙇
...,...,...,...,...,...,...
87605,19219,成功大學,7,=====此封信件,是,由【衛生保健組】寄出，如您想
87606,19220,成功大學,5,今天看到這個實在,是,有些感觸
87607,19220,成功大學,5,,是,看到新聞才知道的
87608,19220,成功大學,5,對他的影響,是,在一些過去電影裡或戲劇中能看到


> 資料筆數多（87610 筆），但是看不到全部 >>> 有更好的呈現方式嗎？互動的可能性？

### 5-4-3: [Data Table Display](https://colab.research.google.com/notebooks/data_table.ipynb)  

- 互動性呈現：篩選、分類、分頁設定等
- 可客製化

In [51]:
# 加入 Data Table Display extension
# renders pandas dataframes into interactive displays that can be filtered, sorted, and explored dynamically

%load_ext google.colab.data_table

In [52]:
pd.DataFrame(result_nuexin)

#TODO: 選出按讚數為 50~100 之間的資料

#TODO: 找出包含「...」、「》」、「〖 〗」的資料
### 解答：使用 regex (\...|〖|〗|》)

,article_id,left,keyword,right,forumName,likeCount
0,5777,「太寫實太,虐心,了吧？」,色情漫畫,93
1,5915,看到結局的地方 太,虐心,太深刻,戲劇綜藝,148
2,5939,最,虐心,的地方就是他與上官曦被綁在,戲劇綜藝,124
3,5965,三世十里桃花》當年讓人如癡如醉的,虐心,指數，從一年半前在橫店開拍,戲劇綜藝,39
4,8369,段交給韓國來拍，絕對會非常的,虐心,，可能不亞於〖與神同行〗。「請,歐美影集,37
5,10506,甚至更長...)就不用這麼,虐心,了,彩虹,56
6,12021,超推！劇情緊張刺激又,虐心,，而且第二季的女主角也太正,電影,128
7,12223,他們的愛情好,虐心,😭,閒聊,2
8,14607,，看到一半以後感覺每一集都在,虐心,,戲劇綜藝,16
9,17410,可以翻一翻我們當初選擇在一起時多,虐心,哭了多少回。,Les,121


In [53]:
# 直接生成 data tables 
from google.colab import data_table

# 可使用參數達到更多客製化的呈現
data_table.DataTable(pd.DataFrame(result_nuexin, columns=columns_order), num_rows_per_page=2)

,article_id,forumName,likeCount,left,keyword,right
0,5777,色情漫畫,93,「太寫實太,虐心,了吧？」
1,5915,戲劇綜藝,148,看到結局的地方 太,虐心,太深刻
2,5939,戲劇綜藝,124,最,虐心,的地方就是他與上官曦被綁在
3,5965,戲劇綜藝,39,三世十里桃花》當年讓人如癡如醉的,虐心,指數，從一年半前在橫店開拍
4,8369,歐美影集,37,段交給韓國來拍，絕對會非常的,虐心,，可能不亞於〖與神同行〗。「請
5,10506,彩虹,56,甚至更長...)就不用這麼,虐心,了
6,12021,電影,128,超推！劇情緊張刺激又,虐心,，而且第二季的女主角也太正
7,12223,閒聊,2,他們的愛情好,虐心,😭
8,14607,戲劇綜藝,16,，看到一半以後感覺每一集都在,虐心,
9,17410,Les,121,可以翻一翻我們當初選擇在一起時多,虐心,哭了多少回。


### 5-4-4: 天哪，我把資料變好看了！ 

#### 玩美步驟

1. 在網頁上按右鍵 
2. 選擇「檢查」 來看 html 程式碼 
3. 找到我們的目標區段（css selector）
4. 寫 css code 來改變「對齊方式」、「顏色」
5. 資料更好看了！（得到美美的 corpus 資料呈現格式）

html 圖解

<a href="https://drive.google.com/file/d/1IeTnQ8yRptYu60Naaj7iZeBbJq-eOd0o/view?usp=sharing" target="_blank"><img src="https://drive.google.com/u/0/uc?id=1IeTnQ8yRptYu60Naaj7iZeBbJq-eOd0o&export=download" width="80%"></a>

In [54]:
# TODO: 利用 html, css selector 來改變資料呈現 (ref. slide page 8)

from IPython.display import HTML, display

def set_css_in_cell_output():
  display(HTML("""<style>
    .google-visualization-table-table  td:nth-child(5) {
      text-align: right;
    } 

    .google-visualization-table-table  td:nth-child(6) {
      text-align: center;
      color: red;
    }

    .google-visualization-table-table  td:nth-child(7) {
      text-align: left;
    }

    
  </style>
  """))

get_ipython().events.register('pre_run_cell', set_css_in_cell_output)

In [55]:
pd.DataFrame(result_nuexin, columns=columns_order)

,article_id,forumName,likeCount,left,keyword,right
0,5777,色情漫畫,93,「太寫實太,虐心,了吧？」
1,5915,戲劇綜藝,148,看到結局的地方 太,虐心,太深刻
2,5939,戲劇綜藝,124,最,虐心,的地方就是他與上官曦被綁在
3,5965,戲劇綜藝,39,三世十里桃花》當年讓人如癡如醉的,虐心,指數，從一年半前在橫店開拍
4,8369,歐美影集,37,段交給韓國來拍，絕對會非常的,虐心,，可能不亞於〖與神同行〗。「請
5,10506,彩虹,56,甚至更長...)就不用這麼,虐心,了
6,12021,電影,128,超推！劇情緊張刺激又,虐心,，而且第二季的女主角也太正
7,12223,閒聊,2,他們的愛情好,虐心,😭
8,14607,戲劇綜藝,16,，看到一半以後感覺每一集都在,虐心,
9,17410,Les,121,可以翻一翻我們當初選擇在一起時多,虐心,哭了多少回。


In [98]:
# TODO: 自己動手玩看看～
#        1. 讀取真實的範例資料 (火大-16, 全家-72, 有事-97, 難過-541)
#        2. 試試看讓每一頁呈現五筆資料就好
#        3. 改變 html 呈現：變更 keyword 欄位的顏色、字型或字體大小；調整所有欄位的對齊方式為置中。

result_sad = query_a_word("難過", window_size=10, show_pos=False, metadata=["forumName", "likeCount"])

a = data_table.DataTable(pd.DataFrame(result_sad, columns=columns_order), num_rows_per_page=5)

def set_css_in_cell_output():
  display(HTML("""<style>
      .google-visualization-table-table  td:nth-child(6) {
        text-align: center;
        color: red;
        font-weight: 800;
        font-family: "微軟正黑體";
    }
  </style>
  """))

get_ipython().events.register('pre_run_cell', set_css_in_cell_output)

a

,article_id,forumName,likeCount,left,keyword,right
0,198,韓星,380,這首歌明明不悲傷 但卻很,難過,
1,198,韓星,380,明明已經很,難過,了 還在台上跟大家有說有笑
2,198,韓星,380,,難過,時就盡量哭
3,227,韓星,52,她私底下,難過,時別人就一定看得到嗎？
4,234,韓星,285,是件很幸福的事，不管開心或,難過,彼此互相成長，在他們身上得到很多
...,...,...,...,...,...,...
536,18947,戲劇綜藝,73,畢竟前幾集鋪陳韻如那麼,難過,壓抑的生活
537,19073,BTS,4,我看到這個是有點,難過,啦！
538,19104,App,0,到底為什麼會這樣，太,難過,了吧
539,19131,寵物,48,希望我不要再為了妳傷心,難過,，希望我放下妳...


## 5-5: 要查詢單詞，也要查詢單個詞性

請試著根據 5-3 所寫的 `query_a_word()`，寫一個 `query_a_token()`，讓這個函數既能查詢單一個word，也能查詢單一個pos。

執行範例：
```python
# 找出所有詞目為「虐心」的token
query_a_token(token_value="虐心", token_type="word", window_size=10, show_pos=False, metadata=["forumName", "likeCount"])

# 找出所有詞性為「VA」的token
query_a_token(token_value="VA", token_type="pos", window_size=10, show_pos=False, metadata=["forumName", "likeCount"])
```

In [99]:
# 注意: query_a_token() 跟剛才的 query_a_word() 只有兩個不同的小地方
#      1. query_a_token() 的前兩個參數變成 token_value 和 token_type
#      2. filter_a_word(word) 變成了 filter_a_token(token_value, token_type)
# TODO: 請大家完成 filter_a_token(token_value, token_type) 的內容

def query_a_token(token_value, token_type, window_size=5, show_pos=False, metadata=[]):

    results = []

    for locator in filter_a_token(token_value, token_type):
        result = generate_concordance_line(locator, window_size, show_pos, metadata)
        results.append(result)

    return results


def filter_a_token(token_value, token_type):

    # 提示: 先判斷 token_type 為何，才知道要使用哪個 index
    index_to_use = None

    # 判斷token_type為何者
    if token_type == "word":
        index_to_use = word_index
    elif token_type == "pos":
        index_to_use = pos_index
    else:
        raise ValueError('參數 token_type 只能是 "word" 或者 "pos"')

    return index_to_use[token_value]

In [100]:
# TODO: 自由時間，填入你想查的token吧，也嘗試看看修改不同參數的結果
result_shayan = query_a_word("傻眼", window_size=10, show_pos=True, metadata=["forumName", "likeCount"])
pd.DataFrame(result_shayan, columns=columns_order)

,article_id,forumName,likeCount,left,keyword,right
0,13,穿搭,324,當下/Nd表情/Na太/Dfa,傻眼/VH,了/Di直接/VH先碼/Na掉哈/VA哈哈/D哈哈/D
1,623,品酒,64,/WHITESPACE上/Nes次/Nf隔壁桌/Na的/DE還/D吐/VC /WHITES...,傻眼/VH,到/P爆/VH
2,770,旅遊,31,不/D知道/VK該/D說/VE,傻眼/VH,還是/Caa他/Nh真材實料/VH，/COMMACATEGORY不過/Cbb喝起來/D是/S...
3,789,旅遊,52,,傻眼/VH,欸/I！/EXCLAMATIONCATEGORY不/D是/SHI應該/D是/SHI你們/Nh...
4,797,旅遊,20,整/Neqa個/Nf,傻眼/VH,，/COMMACATEGORY而且/Cbb太陽/Na底下/Ncd有夠/Dfa熱/VHC
...,...,...,...,...,...,...
206,18147,西斯文學,118,進去/VA後/Ng我/Nh,傻眼/VH,了/T，/COMMACATEGORY他/Nh不/D是/SHI自己/Nh來/VA韓國/Nc的/...
207,18240,西斯文學,53,，/COMMACATEGORY每/Nes次/Nf看到/VE你/Nh莫名其妙/VH有/V_2反...,傻眼/VH,嚇到/VJ
208,18282,西斯文學,71,看/VC著/DiJack/FW虛脫/VH,傻眼/VH,的/DE表情/Na /WHITESPACE喝/VC了/Di口/Nf綠茶/Na
209,18375,臺灣大學,24,我/Nh,傻眼/VH,/WHITESPACE他/Nh期末/Nd測驗/Na排/VC了/Di三/Neu週/Nf欸/I


## 5-6: 在查詢單個token的基礎上，加一點regular expression

請試著根據 5-5 所寫的 `query_a_token()`，寫一個 `query_a_regex_token()`，讓我們可以用 regular expression 查詢單一個word，也能查詢單一個pos。

用處：
- 找出所有以「者」為後綴的詞。
- 找出所有以「老」為前綴的詞。
- 找出所有動詞性的詞。

執行範例：
```python
# 找出所有以「者」為後綴的詞
query_a_regex_token(token_value=".+者$", token_type="word", window_size=10, show_pos=False, metadata=["forumName", "likeCount"])

# 找出所有以「老」為前綴的詞
query_a_regex_token(token_value="^老.+", token_type="word", window_size=10, show_pos=False, metadata=["forumName", "likeCount"])

# 找出所有被標為動詞的詞
query_a_regex_token(token_value="^V.+", token_type="pos", window_size=10, show_pos=False, metadata=["forumName", "likeCount"])
```

In [101]:
# 注意: query_a_regex_token() 跟剛才的 query_a_token() 只有一個不同點
#      1. filter_a_token(token_value, token_type) 變成了 filter_a_regex_token(token_value, token_type)
# TODO: 請大家完成 filter_a_regex_token(token_value, token_type) 的內容

import re

def query_a_regex_token(token_value, token_type, window_size=5, show_pos=False, metadata=[]):

    results = []

    for locator in filter_a_regex_token(token_value, token_type):
        result = generate_concordance_line(locator, window_size, show_pos, metadata)
        results.append(result)

    return results


def filter_a_regex_token(token_value, token_type):

    results = []

    # 提示: 先判斷 token_type 為何，才知道要使用哪個 index
    index_to_use = None

    # 根據不同的token_type，使用相應的 index
    if token_type == "word":
        index_to_use = word_index
    elif token_type == "pos":
        index_to_use = pos_index
    else:
        raise ValueError('參數 token_type 只能是 "word" 或者 "pos"')

    # 開始找出所以符合 token_value 的那些 token
    matched_tokens = [token for token in index_to_use.keys() if re.match(token_value, token)]
    
    for token in matched_tokens:
        for locator in index_to_use[token]:
            results.append(locator)
    
    return results



In [102]:
# TODO: 試著找看看所有以「者」為後綴的詞
result_zhe = query_a_regex_token(token_value=".+者$", token_type="word", window_size=10, show_pos=True, metadata=["forumName", "likeCount"])
pd.DataFrame(result_zhe, columns=columns_order)

,article_id,forumName,likeCount,left,keyword,right
0,16,穿搭,92,/WHITESPACE(/PARENTHESISCATEGORYEnglish /FWNa...,舞者/Na,。/PERIODCATEGORY
1,198,韓星,380,她/Nh讓/VL,舞者/Na,選/VC今天/Nd要/D面向/VJ哪/Nep邊/Ncd唱/VC
2,198,韓星,380,那/Nep天/Nf,舞者/Na,選擇/VC了/Di面向/Na前面/Ncd（/PARENTHESISCATEGORY看/VC吧...
3,2460,BTS,1136,他/Nh接着/VC用/P,舞者/Na,的/DE名義/Na邀請/VC了/Di世界/Nc各/Nes地/Na的/DEdancer/FW來...
4,2475,BTS,96,群/Nf陪/VC著/Di防彈/VH揮灑/VC著/Di汗水/Na和/Caa淚水/Na的/DE,舞者/Na,們/Na，/COMMACATEGORY我/Nh也/D相信/VK彈/VC們/Na一次次/Neq...
...,...,...,...,...,...,...
2831,17975,靈異,5,乳流/Na，/COMMACATEGORY飽足/VH餓/VH鬼眾/Na，/COMMACATEG...,聖者/Na,觀世音/Nb菩薩/Na，/COMMACATEGORY透過/P手/Na上/Ncd淨瓶/Na遍/...
2832,18038,靈異,11,令/VL得/DE安穩/VH，/COMMACATEGORY,離眾者/Na,患/VJ。/PERIODCATEGORY
2833,18927,建議回饋,722,的/DE議題/Na請/VF另行/D發表/VC新/VH文章/Na//FW留言/Na，/COMM...,違者/Na,刪文/VA並/Cbb視/P情況/Na停權/VA 30/Neu /WHITESPACE天/Nf...
2834,18951,電影,440,,違者/Na,將/D刪除/VC文章/Na，/COMMACATEGORY累犯/Na將/D停權/VA 30/N...


In [103]:
# TODO: 找出所有以「老」開頭的 token
result_lao = query_a_regex_token(token_value="^老.+", token_type="word", window_size=10, show_pos=True, metadata=["forumName", "likeCount"])
pd.DataFrame(result_zhe, columns=columns_order)

,article_id,forumName,likeCount,left,keyword,right
0,16,穿搭,92,/WHITESPACE(/PARENTHESISCATEGORYEnglish /FWNa...,舞者/Na,。/PERIODCATEGORY
1,198,韓星,380,她/Nh讓/VL,舞者/Na,選/VC今天/Nd要/D面向/VJ哪/Nep邊/Ncd唱/VC
2,198,韓星,380,那/Nep天/Nf,舞者/Na,選擇/VC了/Di面向/Na前面/Ncd（/PARENTHESISCATEGORY看/VC吧...
3,2460,BTS,1136,他/Nh接着/VC用/P,舞者/Na,的/DE名義/Na邀請/VC了/Di世界/Nc各/Nes地/Na的/DEdancer/FW來...
4,2475,BTS,96,群/Nf陪/VC著/Di防彈/VH揮灑/VC著/Di汗水/Na和/Caa淚水/Na的/DE,舞者/Na,們/Na，/COMMACATEGORY我/Nh也/D相信/VK彈/VC們/Na一次次/Neq...
...,...,...,...,...,...,...
2831,17975,靈異,5,乳流/Na，/COMMACATEGORY飽足/VH餓/VH鬼眾/Na，/COMMACATEG...,聖者/Na,觀世音/Nb菩薩/Na，/COMMACATEGORY透過/P手/Na上/Ncd淨瓶/Na遍/...
2832,18038,靈異,11,令/VL得/DE安穩/VH，/COMMACATEGORY,離眾者/Na,患/VJ。/PERIODCATEGORY
2833,18927,建議回饋,722,的/DE議題/Na請/VF另行/D發表/VC新/VH文章/Na//FW留言/Na，/COMM...,違者/Na,刪文/VA並/Cbb視/P情況/Na停權/VA 30/Neu /WHITESPACE天/Nf...
2834,18951,電影,440,,違者/Na,將/D刪除/VC文章/Na，/COMMACATEGORY累犯/Na將/D停權/VA 30/N...


## 5-7: 限定某詞性下的某詞 (回家練習)

在中文當中，一個詞目可能會有不同的詞性，像是「花」錢和茉莉「花」。請寫一個函數，限定某個詞性下的詞。

## 5-8: [進階] 搜尋多個詞並列的 Phrase query

目前前面所實作的 `query_a_word()`, `query_a_token()` 和 `query_a_regex_token()` 都只能搜索單一個token。

在搜尋語料時，我們可能還會需要針對多個並列的 token 進行搜索，例如 `query_phrase(["做", "一", "個"])`。

所以我們現在要開始實作多個token並列的情況。

執行範例：
```python
# 找出所有 「做」「一」「個」的句子

phrase = [
    {"type": "word", "value": "做"},
    {"type": "word", "value": "一"},
    {"type": "word", "value": "個"},
]

query_phrase(tokens=phrase, window_size=10, show_pos=False, metadata=["forumName", "likeCount"])
```

In [104]:
def query_phrase(tokens, window_size=5, show_pos=False, metadata=[]):

    results = []

    for locator in filter_phrase(tokens):
        result = generate_concordance_line_v2(locator, window_size, show_pos, metadata)
        results.append(result)

    return results

In [105]:
def filter_phrase(tokens):

    results = []
    token_freqs = []

    # 先比較傳進來的 list of tokens，哪一個 token 的出現次數最少 => 為了對搜尋做優化
    for token in tokens:
        if token["type"] == "word":
            # 使用我們前面已經寫好的 word_freq 函數
            token_freq = word_freq(token["value"])

        elif token["type"] == "pos":
            token_freq = 1000000000
            # token_freq = sum([pos_freq(pos) for pos in pos_index.keys() if re.match(token["value"], pos)])
            # len(pos_index[token["value"]])
        else:
            raise ValueError("token type只能是 word 或 pos")

        token_freqs.append(token_freq)

    # 找出 token_freqs 最小的
    min_freq = min(token_freqs)

    # 找出出現次數最少的 token 是的第幾個
    min_freq_idx = token_freqs.index(min_freq)

    # 找出出現次數最少的 token value
    min_freq_token_value = tokens[min_freq_idx]["value"]
    
    # 出現次數最少的 token 的位置離tokens 最左邊的元素多遠?
    normalized_index = [i - min_freq_idx for i in range(len(tokens))]

    # 從出現次數最少次的 token 開始找
    for a_idx, s_idx, t_idx in word_index[min_freq_token_value]:
        
        need_to_pass = False
        sentence = corpus[a_idx]["text"][s_idx]

        for i in range(len(tokens)):
            if i != min_freq_idx:
                # 因為可能會超過句子邊界，所以使用 try...except...處理這種情況
                try:
                    if tokens[i]["type"] == "word":
                        if sentence[t_idx + (i - min_freq_idx)][tokens[i]["type"]] != tokens[i]["value"]:
                            need_to_pass = True
                            break
                    elif tokens[i]["type"] == "pos":
                        if not re.match(tokens[i]["value"], sentence[t_idx + (i - min_freq_idx)][tokens[i]["type"]]):
                            need_to_pass = True
                            break

                except:
                    need_to_pass = True
                    break
        if need_to_pass:
            continue

        results.append([a_idx, s_idx, [i + t_idx for i in normalized_index]])
    
    return results

In [106]:
# 現在 locator 傳進來的東西是 [10, 2, [20, 21, 22]]
# 也就是說 t_idx 不再是單個整數，而是list of int
def generate_concordance_line_v2(locator, window_size, show_pos, metadata):
    a_idx = locator[0]
    s_idx = locator[1]
    t_idxs = locator[2]

    sentence = corpus[a_idx]['text'][s_idx]

    # 準備好每個元素
    each_concordance_line = {
        "article_id": a_idx,
        "left": "",
        "keyword": "",
        "right": ""
    }

    # 需要哪些metadata，就把那些metadata裝進來
    for key in metadata:
        each_concordance_line[key] = corpus[a_idx][key]

    # 判斷是否要顯示詞性
    if show_pos:
        each_concordance_line["keyword"] = " ".join([sentence[t_idx]["word"] + "/" + sentence[t_idx]["pos"] for t_idx in t_idxs])
    else:
        each_concordance_line["keyword"] = " ".join([sentence[t_idx]["word"] for t_idx in t_idxs])

    # 安全邊界
    safe_left_bound = max(0, t_idxs[0] - window_size)
    safe_right_bound = min(len(sentence), t_idxs[-1] + window_size + 1)


    # 先處理處理關鍵字的左邊部分
    for token in sentence[safe_left_bound: t_idxs[0]]:
        if show_pos:
            each_concordance_line["left"] += token["word"] + "/" + token["pos"]
        else:
            each_concordance_line["left"] += token["word"]

    # 再處理處理關鍵字的右邊部分
    for token in sentence[t_idxs[-1] + 1: safe_right_bound]:
        if show_pos:
            each_concordance_line["right"] += token["word"] + "/" + token["pos"]
        else:
            each_concordance_line["right"] += token["word"]

    return each_concordance_line

In [107]:
# TODO: 請找出所有 "的" "動作" 的句子
phrase = [
    {"type": "word", "value": "的"},
    {"type": "word", "value": "動作"},
]
 
result_de_dongzuo = query_phrase(tokens=phrase, window_size=10, show_pos=True, metadata=["forumName", "likeCount"])
pd.DataFrame(result_de_dongzuo, columns=columns_order)

,article_id,forumName,likeCount,left,keyword,right
0,28,穿搭,13134,會/D選/VC這/Nep張/Nf是/SHI因為/Cbb我/Nh覺得/VK比/VC,的/DE 動作/Na,很/Dfa可愛/VH☺/FW️/Na
1,468,動漫,20,在/P跑步/VA、/PAUSECATEGORY隊友/Na連接/VJ還/D有/V_2過/Di障...,的/DE 動作/Na,我/Nh覺得/VK都/D挺/Dfa流暢/VH的/DE
2,505,動漫,18,在/P某/Nes個/Nf畫面/Na比較/Dfa久/VH，/COMMACATEGORY或是/C...,的/DE 動作/Na,卻/D好像/D少/VH了/Di中間/Ncd的/DE過程/Na
3,622,品酒,10,次/Nf格蘭路思/Nb選擇/VC全部/Neqa以/P美國/Nc橡木桶/Na來/D進行/VC熟...,的/DE 動作/Na,，/COMMACATEGORY更/Dfa與/P2018年/Nd造成/VK全球/Nc市場/Nc...
4,655,品酒,56,練習/VC 量/Na酒器/Na /WHITESPACE加/VC冰/Na /WHITESPAC...,的/DE 動作/Na,
...,...,...,...,...,...,...
190,18292,西斯文學,59,她/Nh欺負/VC的/DE花紅/VH的/DE肩/Na上/Ncd，/COMMACATEGORY...,的/DE 動作/Na,沒有/D停/VHC反而/Cbb更/D強硬/VH了/Di一些/Dfb。/PERIODCATEGORY
191,18941,戲劇綜藝,37,再/D加上/Cbb總是/D用/P無害/VH的/DE臉/Na做超撩妹/VA,的/DE 動作/Na,
192,18960,語言,0,./PERIODCATEGORY./PERIODCATEGORY./PERIODCATEGO...,的/DE 動作/Na,名稱/Na
193,18977,汽機車,10,說/VE：/COLONCATEGORY「/PARENTHESISCATEGORY我們/Nh這...,的/DE 動作/Na,」/PARENTHESISCATEGORY


## 5-9: 比較 `V下來` 和 `V下去`

In [108]:
# TODO: 請找出所有 V + "下來" 的句子
phrase = [
    {"type": "pos", "value": "V.+"},
    {"type": "word", "value": "下來"},
]
 
result_V_xialai = query_phrase(tokens=phrase, window_size=10, show_pos=True, metadata=["forumName", "likeCount"])
pd.DataFrame(result_V_xialai, columns=columns_order)

,article_id,forumName,likeCount,left,keyword,right
0,357,省錢,20,這樣/VH,比較/VC 下來/Di,還/D蠻/Dfa優惠/VJ的/DE
1,385,省錢,9,,使用/VC 下來/Di,比/PEZtable/FW便宜/VH5/Neu塊/Nf 比/FW原價/Na便宜/VH88/N...
2,670,品酒,40,倒進/VCL殺菌/VA過/Di的/DE玻璃罐/Na密封/VB，/COMMACATEGORY倒...,冷卻/VHC 下來/Di,之後/Ng放進/VC冰箱/Na，/COMMACATEGORY就/D完成/VC了/Di～/FW
3,683,品酒,10,-/DASHCATEGORY波摩/Nb三十六/Neu年/Nf的/DE價格/Na為/VG150...,換算/VC 下來/Di,為/VG六萬六千/Neu塊/Nf錢/Na台幣/Na，/COMMACATEGORY在/P台灣/...
4,694,品酒,21,了/Di85%/Neqa有機/A黑/VH巧克力/Na與/Caa桂圓/Na核桃/Na奶油餅/N...,搭配/VC 下來/Di,覺得/VK桂圓/Na合桃/Na奶油餅/Na很/Dfa搭/VC，/COMMACATEGORY核...
...,...,...,...,...,...,...
112,18198,西斯文學,24,福爾摩斯/Nb目光/Na,暗沉/VH 下來/Di,，/COMMACATEGORY把玩/VC著/Di她/Nh的/DE長髮/Na，/COMMACA...
113,18544,遊戲,1,看到/VE☀️/Na,閃耀/VH 下來/Di,心臟/Na真的/D要/D停/VHC了/Di_/FW:/COLONCATEGORY(/PARE...
114,18634,Apple,2,如果/Cbb,換算/VC 下來/Di,的話/Cba大概/D三/Neu分鐘/Nf就/D掉/VH1%/Neqa的/DE電/Na
115,19028,醫美,7,後來/Nd好不容易/D要/D,放鬆/VHC 下來/Di,


In [109]:
# TODO: 請找出所有 V + "下去" 的句子

phrase = [
    {"type": "pos", "value": "V.+"},
    {"type": "word", "value": "下去"},
]
 
result_V_xiaqu = query_phrase(tokens=phrase, window_size=10, show_pos=True, metadata=["forumName", "likeCount"])
pd.DataFrame(result_V_xiaqu, columns=columns_order)

,article_id,forumName,likeCount,left,keyword,right
0,192,韓星,82,要/D繼續/VF這樣/VH,延續/VJ 下去/Di,
1,455,動漫,12,會/D有/V_2股/Nf魔力/Na，/COMMACATEGORY會/D讓/VL人/Na想/V...,觀看/VC 下去/Di,，/COMMACATEGORY想/VE看/VC後續/A角色/Na的/DE發展/VC，/COM...
2,517,動漫,51,但/Cbb還是/D期待/VK萌王/Na會/D如何/D,開掛/VC 下去/Di,XDDD/FW
3,523,動漫,69,其他/Neqa就/D請/VF各位/Nh自己/Nh去/D看/VC吧/T，/COMMACATEG...,說/VE 下去/Di,就/D沒完沒了/VH┐/PARENTHESISCATEGORY(/PARENTHESISCA...
4,592,品酒,34,了/T，/COMMACATEGORY秉持/VJ著/Di學習/VC的/DE心態/Na，/COM...,花/VC 下去/Di,了/T，/COMMACATEGORY但/Cbb要/D買/VC整/Neqa瓶/Nf的話/Cba...
...,...,...,...,...,...,...
315,18825,心情,1572,然後/D在/P學長/Na的/DE,陪同/VC 下去/Di,找/VC主辦/VC方/D做/VC處理/VC
316,18882,美甲,16,還/D有/V_2些許/Neqa的/DE小/VH,鑽/VC 下去/Di,點綴/VC
317,18954,電影,11,真實/VH的/DE影響/Na」/PARENTHESISCATEGORY，/COMMACATE...,傳誦/VJ 下去/Di,。/PERIODCATEGORY
318,18973,汽機車,19,什麼/Nep，/COMMACATEGORY因為/Cbb比較/Dfa好/VH整理/VC，/CO...,更新/VC 下去/Di,


In [110]:
# 自由發揮看看
phrase = [
    {"type": "pos", "value": "V.+"},
    {"type": "word", "value": "東"},
    {"type": "pos", "value": "V.+"},
    {"type": "word", "value": "西"},
]
 
result_V_lai_V_qu = query_phrase(tokens=phrase, window_size=10, show_pos=True, metadata=["forumName", "likeCount"])
pd.DataFrame(result_V_lai_V_qu, columns=columns_order)

,article_id,forumName,likeCount,left,keyword,right
0,573,動漫,345,折腰/VA，/COMMACATEGORY接受/VC上/Ng至/P斬妖/VA除魔/VA，/CO...,修/VC 東/Ncd 修/VC 西/Ncd,的/DE各/Nes類/Nf委託/VF。/PERIODCATEGORY
1,899,精品,11,，/COMMACATEGORY皮夾/Na根本/D就/D放不下/VJ，/COMMACATEGO...,掉/VH 東/Ncd 掉/VH 西/Ncd,的/DE😂😂/Na
2,3850,居家生活,107,前/Ng的/DE事/Na了/T，/COMMACATEGORY今天/Nd不/D就事論事/VA又...,來/VA 東/Ncd 扯/VC 西/Ncd,扯/VC什麼/Nep？/QUESTIONCATEGORY
3,4086,音樂,2,總是/D,想/VE 東/Ncd 想/VE 西/Ncd,怎麼樣/VH都/D吃/VC不/D好/VC睡/VA不/D好/VH
4,5098,理財,31,我/Nh要/D買/VC什麼/Nep比較/VC高額/Na的/DE東西/Na都/D會/D,這樣/VH 東/Ncd 問/VE 西/Ncd,問/VE啦/T）/PARENTHESISCATEGORY
5,6702,感情,13,東/Ncd拼/VC西/Ncd湊/VC後/Ng /WHITESPACE故事/Na還是/D很/D...,順😭/VH 東/Ncd 拼/VC 西/Ncd,湊/VC後/Ng /WHITESPACE故事/Na還是/D很/Dfa不/D順😭/VH
6,8973,靈異,18,用人/VA的/DE智慧/Na去/D想像/VK。/PERIODCATEGORY我們/Nh在/P...,爭/VC 東/Ncd 搶/VD 西/Ncd,的/T，/COMMACATEGORY有/V_2什麼/Nep意義/Na呢/T？/QUESTIO...
7,9064,西斯文學,34,「/PARENTHESISCATEGORY因為/Cbb妳/Nh,想/VE 東/Ncd 想/VE 西/Ncd,啊/T。/PERIODCATEGORY有/V_2什麼/Nep不/D如意/VH的/DE事/Na...
8,9493,護理,588,她/Nh單位/Na，/COMMACATEGORY現在/Nd人/Na夠/VH了/Di就/D在/...,說/VE 東/Ncd 說/VE 西/Ncd,，/COMMACATEGORY她/Nh當初/Nd面試/VC說/VE有/V_2什麼/Nep問題...
9,9539,護理,119,,送/VD 東/Ncd 送/VD 西/Ncd,是/SHI小/VH事/Na


# Step 6: Collocation

![](https://static.cambridge.org/binary/version/id/urn:cambridge.org:id:binary:20180904072708260-0799:9781316410899:12570tbl3_1.png?pub-status=live)

![](https://static.cambridge.org/binary/version/id/urn:cambridge.org:id:binary:20180904072708260-0799:9781316410899:12570tbl3_2.png?pub-status=live)

各種Association measure:

![](https://static.cambridge.org/binary/version/id/urn:cambridge.org:id:binary:20180904072708260-0799:9781316410899:12570tbl3_3.png?pub-status=live)

## 6-1: 看「造成」一詞右邊一格的Collocation

右邊一格的意思就是 window size 為 1R (one 有時也能標為 +1)

在這個練習我們將以 LogDice 這個 association measure 為例子。

In [111]:
# LogDice 算式中只出現 O11, R1, C1

## 以 造成 + 困擾 為例
### O11 為 造成 + 困擾 的總數
### R1  為 造成 + X    的總數 ~ 先以造成的總數來看
### C1  為 X   + 困擾  的總數 ~ 先以困擾的總數來看

# 預期的 result 結果:
# {
#    "困擾": {"O11": <'造成困擾'出現次數>, "R1": <'造成'詞頻>, "C1": <'困擾'詞頻>},
#    "改變": {"O11": <'造成改變'出現次數>, "R1": <'造成'詞頻>, "C1": <'改變'詞頻>},
#    ...
# }

result = {}

# 「造成」的詞頻
R1 = len(word_index['造成'])

for a_idx, s_idx, w_idx in word_index['造成']:
    
    sentence_with_造成 = corpus[a_idx]['text'][s_idx]

    # 跳過 造成 在句尾的情況
    if w_idx + 1 >= len(sentence_with_造成):
      continue

    # word_1r 為在這句話中，出現在「造成」後面的那個詞
    word_1r = corpus[a_idx]['text'][s_idx][w_idx + 1]['word']

    # 要是 word_1r 尚未出現在 result 之中，就初始化
    if word_1r not in result:
      result[word_1r] = {"O11": 1, "C1": len(word_index[word_1r]), "R1": R1}
    
    # 要是 word_1r 已經出現過了，那麼只要把他的 O11 加上 1 就好
    else:
      result[word_1r]["O11"] += 1

In [112]:
result

{' ': {'C1': 54959, 'O11': 2, 'R1': 510},
 '   ': {'C1': 394, 'O11': 1, 'R1': 510},
 ' 58': {'C1': 1, 'O11': 1, 'R1': 510},
 ' Bee’s Knees ': {'C1': 2, 'O11': 1, 'R1': 510},
 ',': {'C1': 4202, 'O11': 1, 'R1': 510},
 '.': {'C1': 44611, 'O11': 1, 'R1': 510},
 '2020': {'C1': 669, 'O11': 1, 'R1': 510},
 '285': {'C1': 3, 'O11': 1, 'R1': 510},
 '41': {'C1': 35, 'O11': 2, 'R1': 510},
 '73': {'C1': 27, 'O11': 1, 'R1': 510},
 'KD': {'C1': 30, 'O11': 1, 'R1': 510},
 'and one': {'C1': 2, 'O11': 1, 'R1': 510},
 'x': {'C1': 331, 'O11': 1, 'R1': 510},
 '、': {'C1': 28877, 'O11': 1, 'R1': 510},
 '。': {'C1': 53661, 'O11': 1, 'R1': 510},
 '「': {'C1': 17717, 'O11': 1, 'R1': 510},
 '一': {'C1': 51249, 'O11': 5, 'R1': 510},
 '一些': {'C1': 3306, 'O11': 3, 'R1': 510},
 '一時': {'C1': 135, 'O11': 1, 'R1': 510},
 '下載': {'C1': 350, 'O11': 1, 'R1': 510},
 '不': {'C1': 49774, 'O11': 6, 'R1': 510},
 '不便': {'C1': 29, 'O11': 2, 'R1': 510},
 '不悅': {'C1': 31, 'O11': 1, 'R1': 510},
 '不錯': {'C1': 1685, 'O11': 1, 'R1': 510},


In [113]:
result_df = pd.DataFrame.from_dict(result, orient='index')
result_df

,O11,C1,R1
大家,7,13318,510
的,63,231504,510
不便,2,29,510
洗版,1,44,510
隔週,1,9,510
...,...,...,...
至少,1,912,510
恐慌,1,49,510
搶購,1,15,510
違規,1,141,510


LogDice = $14+log_2\frac{2 \times O_{11}}{R_1 + C_1}$

In [114]:
#TODO: 寫一個運算 log_dice 的函數
import numpy as np
def log_dice(O11, R1, C1):
  return 14 + np.log(2 * O11 / (R1 + C1))

In [115]:
result_df['log_dice'] = result_df.apply(lambda x: log_dice(x['O11'], x['R1'], x['C1']), axis=1)
result_df

,O11,C1,R1,log_dice
大家,7,13318,510,7.104607
的,63,231504,510,6.481729
不便,2,29,510,9.096579
洗版,1,44,510,8.375982
隔週,1,9,510,8.441243
...,...,...,...,...
至少,1,912,510,7.433328
恐慌,1,49,510,8.366998
搶購,1,15,510,8.429749
違規,1,141,510,8.214638


In [116]:
# TODO: 找自己有興趣的詞來看看